In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fastai.text import *
from pprint import pprint

In [2]:
df = pd.read_csv('/media/winstonfan/Workspace/MyDoc/Interviews/DigitalVictoria/code/COVID-Q/final_master_dataset.csv')

In [3]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

Category    366
Question      1
Answers     763
dtype: int64

## 1. Construct the word dataset for language model

In [4]:
df_word = df[['Question']]

In [5]:
# Check None records
null_columns=df_word.columns[df_word.isnull().any()]
df_word[null_columns].isnull().sum()

Question    1
dtype: int64

In [6]:
df_word = df_word.dropna(subset=['Question'])
df_word.columns=['text']
df_word.tail()

,text
1606,why are churches closed due to covid
1607,is it safe to have play rehearsals during covid
1608,where can i buy toilet paper considering most ...
1609,will i likely catch the covid virus in america...
1610,what are your best tips for entertaining young...


In [9]:
# save the data set for use later
# word_df.to_csv('word_ds.csv', index=False)

### 2. Construct the dataset for text classification model

In [7]:
df_questions = df.dropna(subset=['Question'])
df_questions = df_questions.dropna(subset=['Category'])
df_questions.tail()

,Category,Question ID,Question,Source,Answers
1240,Reporting - Media Coverup,634,is the media hiding what is really going on in...,Quora,NaN
1241,Treatment - Quinine,650,can quinine be taken to fight this covid virus,Quora,NaN
1242,Treatment - Quinine,650,why is quinine effective in curing covid,Quora,NaN
1243,Other - Food Shortages,126,could we run out of things to eat,Author Generated,NaN
1244,Other - Food Shortages,126,do i need to stock up on food,Author Generated,NaN


In [12]:
df_questions = df_questions[['Category', 'Question']]
df_questions.columns = ['label','text']
df_questions.tail()

,label,text
1240,Reporting - Media Coverup,is the media hiding what is really going on in...
1241,Treatment - Quinine,can quinine be taken to fight this covid virus
1242,Treatment - Quinine,why is quinine effective in curing covid
1243,Other - Food Shortages,could we run out of things to eat
1244,Other - Food Shortages,do i need to stock up on food


In [14]:
categories = df_questions.label.unique()
categories.shape

(208,)

In [18]:
categories

array(['Speculation - Pandemic Duration',
       'Transmission - Demographics: General',
       'Transmission - Survivability: Surfaces',
       'Nomenclature - Reason for Name', 'Reporting - Number of Cases',
       'Societal Response - United States',
       'Transmission - Survivability: Temperature',
       'Societal Effects - General', 'Societal Effects - Apocalyptic',
       'Transmission - Food Contamination', 'Origin - Country',
       'Origin - Manufactured in Laboratory',
       'Origin - Planned Release Conspiracy',
       'Transmission - Possibility of Reinfection',
       'Transmission - General', 'Nomenclature - Pandemic Definition',
       'Prevention - Use of Facemask', 'Prevention - General',
       'Prevention - Alternative Disinfection', 'Treatment - Chloroquine',
       'Other - Mutation', 'Origin - Source Animal',
       'Testing - Methodology', 'Comparison - Worse than Flu',
       'Prevention - Time Until Vaccine', 'Reporting - Deaths',
       'Transmission - Sho

In [36]:
categories = np.sort(categories)
categories

array(['Comparison - Black Death', 'Comparison - COVID and HIV',
       'Comparison - COVID versus Flu Diagnosis',
       'Comparison - Difference from Flu',
       'Comparison - Earlier Existence', 'Comparison - Earlier Strains',
       'Comparison - Other Viruses', 'Comparison - Reason for Danger',
       'Comparison - SARS', 'Comparison - Worse than Flu',
       'Economic Effects - Cryptocurrencies',
       'Economic Effects - General',
       "Economic Effects - Individuals' Finances",
       'Economic Effects - Industries', 'Economic Effects - Making Money',
       'Economic Effects - Profitable Stocks',
       'Economic Effects - Real Estate',
       'Economic Effects - Starting a Business',
       'Economic Effects - Stocks',
       'Economic Effects - Technology Industry',
       'Economic Effects - World Superpower',
       'Having COVID - Ability to Recover',
       'Having COVID - Cause of Death', 'Having COVID - Effects on Body',
       'Having COVID - Incubation Period',
 

In [44]:
row = df_questions.iloc[0]
type(row)

pandas.core.series.Series

In [49]:
def getCategoryId(label):
    index = np.where(categories == label)
    # we dont want to have Id as 0
    return index[0].item() + 1

# getCategoryId(df_questions.iloc[0]), df_questions.iloc[0]['label']
df_questions['cid'] = df_questions['label'].apply(lambda label: getCategoryId(label))
df_questions.tail()

,label,text,cid
1240,Reporting - Media Coverup,is the media hiding what is really going on in...,92
1241,Treatment - Quinine,can quinine be taken to fight this covid virus,208
1242,Treatment - Quinine,why is quinine effective in curing covid,208
1243,Other - Food Shortages,could we run out of things to eat,61
1244,Other - Food Shortages,do i need to stock up on food,61


In [51]:
df_questions.drop(['label'], axis=1, inplace=True)
df_questions.columns = ['text', 'label']
df_questions.tail()

,text,label
1240,is the media hiding what is really going on in...,92
1241,can quinine be taken to fight this covid virus,208
1242,why is quinine effective in curing covid,208
1243,could we run out of things to eat,61
1244,do i need to stock up on food,61


In [52]:
df_questions.to_csv('questions_ds.csv', index=False)